Task 2 - Calculate the net value of the contract for any given date of purchase and sell by using data from Nat_gas.csv.

In [2]:
import pandas as pd
import numpy as np
import csv
from matplotlib import pyplot as plt
from datetime import date, timedelta

In [4]:
df = pd.read_csv('Nat_gas.csv', parse_dates=['Dates'])
prices = df['Prices'].values
dates = df['Dates'].values

def get_price_on_date(target_date_str):
    try:
        # Convert the input string to a datetime object
        target_date = pd.to_datetime(target_date_str, format='%m/%d/%Y')
    except ValueError:
        # Handles the case where the date conversion fails
        print("Invalid date format. Please use 'MM/DD/YYYY'.")
        return None
    
    # Find the price for the converted date
    if target_date in df['Dates'].values:
        price = df.loc[df['Dates'] == target_date, 'Prices'].iloc[0]
        return price
    else:
        return None  # Return None if no matching date is found


/var/folders/dw/3r081_b1335dzvqm6llcz3540000gn/T/ipykernel_14552/4069250020.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('Nat_gas.csv', parse_dates=['Dates'])


In [5]:
# function for estimating value of a contract
def calculate_gas_contract_value(volume_mmbtu, purchase_price, sell_price, storage_fee_per_month, 
                                 months_stored, injection_withdrawal_cost, transportation_cost):
    # Purchase cost
    total_purchase_cost = volume_mmbtu * purchase_price
    
    # Revenue from selling the gas
    total_revenue = volume_mmbtu * sell_price

    # Storage cost
    total_storage_cost = storage_fee_per_month * months_stored
    
    # Injection/withdrawal cost
    total_injection_withdrawal_cost = injection_withdrawal_cost*volume_mmbtu
    
    # Transportation cost
    total_transportation_cost = transportation_cost  
    
    # Net contract value
    net_contract_value = (total_revenue - total_purchase_cost - total_storage_cost - 
                          total_injection_withdrawal_cost - total_transportation_cost)
    
    return net_contract_value

In [6]:
# Every considered cost for given situation
def calculate_net_contract_value(purchase_date_str, sell_date_str, desired_volume, max_volume):
    # Costs in current cases
    storage_fee_per_month = 10e4  # [$/month]
    injection_withdrawal_cost = 1e4/1e6  # [$/million MMBtu]
    transportation_cost = 5e4  # [$/transaction]

    volume = min(desired_volume, max_volume)

    purchase_price = get_price_on_date(purchase_date_str)
    sell_price = get_price_on_date(sell_date_str)

    months_stored = (pd.to_datetime(sell_date_str, format='%m/%d/%Y').year - pd.to_datetime(purchase_date_str, format='%m/%d/%Y').year) * 12 + \
                        (pd.to_datetime(sell_date_str, format='%m/%d/%Y').month - pd.to_datetime(purchase_date_str, format='%m/%d/%Y').month)

    contract_value = calculate_gas_contract_value(volume, purchase_price, sell_price, storage_fee_per_month, 
                                                months_stored, injection_withdrawal_cost, transportation_cost)
    
    return contract_value

In [7]:
inject_date_str = '6/30/2023'  # Example date in MM/DD/YYYY format
withdraw_date_str = '2/29/2024'
volume = 1e6  # in MMBtu
max_volume = 2e6 # in MMBtu
result = calculate_net_contract_value(inject_date_str, withdraw_date_str, volume, max_volume)
print(f"The net value of the contract is: ${result:.2f}")


The net value of the contract is: $640000.00
